In [1]:
import pandas as pd
import datetime
import sched, time

In [34]:
CMC_data_last = pd.read_json("https://api.coinmarketcap.com/v1/ticker")
CMC_data_last = CMC_data_last[["rank", "name", "price_usd", "price_btc", "24h_volume_usd", "percent_change_1h", "percent_change_24h"]]
CMC_data_last["time"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
CMC_data_last_price_only = CMC_data_last[["rank", "name", "price_usd", "time"]]
CMC_data_30_ago = CMC_data_last_price_only
CMC_data_25_ago = CMC_data_last_price_only
CMC_data_20_ago = CMC_data_last_price_only
CMC_data_15_ago = CMC_data_last_price_only
CMC_data_10_ago = CMC_data_last_price_only
CMC_data_05_ago = CMC_data_last_price_only

In [37]:
starttime=time.time()
while True:
    CMC_data_30_ago = CMC_data_25_ago
    CMC_data_25_ago = CMC_data_20_ago
    CMC_data_20_ago = CMC_data_15_ago
    CMC_data_15_ago = CMC_data_10_ago
    CMC_data_10_ago = CMC_data_05_ago
    CMC_data_05_ago = CMC_data_last_price_only
    CMC_data_last = pd.read_json("https://api.coinmarketcap.com/v1/ticker")
    CMC_data_last = CMC_data_last[["rank", "name", "price_usd", "price_btc", "24h_volume_usd", "percent_change_1h", "percent_change_24h"]]
    CMC_data_last["time"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    CMC_data_last_price_only = CMC_data_last[["rank", "name", "price_usd", "time"]]
    CMC_data_delta = CMC_data_last_price_only.rename(columns = {'price_usd':'price_00'}).merge(CMC_data_05_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_05'}), on="name").merge(CMC_data_10_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_10'}), on="name").merge(CMC_data_20_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_20'}), on="name").merge(CMC_data_30_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_30'}), on="name")
    CMC_data_delta["percent_change_05m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_05"])/(CMC_data_delta["price_05"]),5)*100
    CMC_data_delta["percent_change_10m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_10"])/(CMC_data_delta["price_10"]),5)*100
    CMC_data_delta["percent_change_20m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_20"])/(CMC_data_delta["price_20"]),5)*100
    CMC_data_delta["percent_change_30m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_30"])/(CMC_data_delta["price_30"]),5)*100
    CMC_data_last = CMC_data_last.merge(CMC_data_delta[["name", "percent_change_05m", "percent_change_10m", "percent_change_20m", "percent_change_30m"]], on="name")

    time.sleep(10.0 - ((time.time() - starttime) % 10.0))

KeyboardInterrupt: 

In [ ]:
CMC_data_last[CMC_data_last["24h_volume_usd"]>1000000].sort_values(by = "percent_change_05m", ascending=False)

In [33]:
df_total = pd.read_json("https://api.coinmarketcap.com/v1/ticker")
df_1h = df_total[["name", "percent_change_1h", "24h_volume_usd"]]
df_1h = df_1h.set_index("name")
print(df_1h.head(2))

          percent_change_1h  24h_volume_usd
name                                       
Bitcoin               -0.17    1.686520e+09
Ethereum              -0.15    5.897790e+08


In [46]:
s = sched.scheduler(time.time, time.sleep)

iteration = 1

def gatherPrices(df, iteration):
    df_total_temp = pd.read_json("https://api.coinmarketcap.com/v1/ticker")
    df_1h_last = df_total_temp[["name", "percent_change_1h"]]
    df_1h_last = df_1h_last.set_index("name")
    if iteration <= 2:
        df_final = df.join(df_1h_last, rsuffix="_"+str(iteration))
    else:
        df = df.join(df_1h_last, rsuffix="_"+str(iteration)) # les multiples de 4 ne s'affichent pas car à chaque
                                                             # multiple de la 3e iteration, on se retrouve avec 3
                                                             # perc_chang_1h_X, du coup le rsuffix ne se rajoute pas                                                          
        df_final = df.drop(df.columns[[0]], axis=1)
    print(df_final[df_final["24h_volume_usd"]>1000000].sort_values(by = df_final.columns[0], ascending=False).head(5))
    iteration += 1 #ça faisait 4-8-16-32 etc ça tu itérait iteration += iteration (exponential). j'ai remplacé par +=1
    s.enter(2, 1, gatherPrices, (df_final,iteration))


s.enter(2, 1, gatherPrices, (df_1h,iteration))
s.run()

             percent_change_1h  24h_volume_usd  percent_change_1h_1
name                                                               
GameCredits              10.14       3580940.0                 6.34
ZenCash                   5.20       1525710.0                -1.93
Clams                     3.75       1544300.0                 3.64
Ubiq                      2.51       1131590.0                 2.40
Riecoin                   2.17       2664900.0                 1.82
             percent_change_1h  24h_volume_usd  percent_change_1h_1  \
name                                                                  
GameCredits              10.14       3580940.0                 6.34   
ZenCash                   5.20       1525710.0                -1.93   
Clams                     3.75       1544300.0                 3.64   
Ubiq                      2.51       1131590.0                 2.40   
Riecoin                   2.17       2664900.0                 1.82   

             percent_chang

KeyError: '24h_volume_usd'